In [ ]:
import os
import requests

from dotenv import load_dotenv
from pathlib import Path

from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel

model = "sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
# env_path = Path('..') / '.env'
# load_dotenv(dotenv_path=env_path)
# hf_token = os.environ.get('HUGGING_FACE_TOKEN');